# WaMDaM_Use_Case 5: Serve Data to WEAP and estimate demand shortage at the Bird Refuge, Utah

#### By Adel M. Abdallah, Utah State University, August 2018

Execute the following cells by pressing `Shift-Enter`, or by pressing the play button 
<img style='display:inline;padding-bottom:15px' src='play-button.png'>
on the toolbar above.


### Steps to reproduce this use case results 

1.[Import python libraries](#Import)   

2.[Connect to the WEAP API](#ConnectWEAP)  

3.[Add Scenarios to WEAP](#AddScenarios)

4.[Connect to the WaMDaM populated SQLite file](#Connect)    
 
----------------------
A. [Query Supply Flow Data and load it to WEAP](#QuerySupplyDataLoadWEAP)

A.1 [Query WaMDaM dababase for flow time series and prepare it for WEAP](#QueryFlowData) 

A.2 [Load the seasonal headflow data into WEAP](#LoadFlow)

----------------------

B. [Query demand data and load it to WEAP ](#QueryandLoadDemand)

B.1 [Query WaMDaM dababase for flow time series and prepare it for WEAP](#QueryDemandData) 

B.2 [Load the seasonal headflow data into WEAP](#LoadDemand)

----------------------


C. [Query Bathymetry data and load it to WEAP](#QueryLoadBathymetry)

C.1 [Create a new WEAP Area for with new Reservoir Bathymetry](#CreateWEAP_Area)

C.2 [Query the multi-column array: Reservoir Bathymetry](#QueryBathymetry) 

C.3 [Load the Bathymetry to WEAP](#LoadBathymetry)

----------------------

   
[Close the SQLite and WEAP API connections](#Close)  



<a name="Import"></a>
# 1. Import python libraries 

In [ ]:
# 1. Import python libraries 
### set the notebook mode to embed the figures within the cell


import plotly
plotly.__version__
import plotly.offline as offline
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
offline.init_notebook_mode(connected=True)
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *

init_notebook_mode(connected=True)         # initiate notebook for offline plot

import os
import csv
from collections import OrderedDict
import sqlite3
import pandas as pd
import numpy as np
from IPython.display import display, Image, SVG, Math, YouTubeVideo
import urllib
from scipy import stats
import calendar

print 'The needed Python libraries have been imported'

<a name="ConnectWEAP"></a>
# 2. Connect to the WEAP API

### You need to have WEAP already installed on your machine

First make sure to have a copy of the Water Evaluation And Planning" system (WEAP) installed on your local machine (Windows). 
You will need to have an active license to use the API
For more info, see here http://www.weap21.org/index.asp?action=40

## WEAP API info 
http://www.weap21.org/WebHelp/API.htm

## Install dependency and register WEAP
### A. Install pywin32 extensions which provide access to many of the Windows APIs from Python.
**Choose on option**
1. Install using an executable basedon your python version. I used Python 2.7
https://github.com/mhammond/pywin32/releases

2. Install from source code (for advanced users) 
https://github.com/mhammond/pywin32

### B. Register WEAP with Windows 


# Install WEAP and Resiger it 


This use case only works on a local Jupyter Notebook server installed on your machine along with WEAP. So it does not work on the online Notebooks in Step 2.1. You need to install Jupyter Server in Step 2.2 then proceed here.

2.1	Install WEAP software. 
If you don’t have it installed, download and install the WEAP software which allows you to run the Bear River WEAP model and its scenarios for Use Case 5. https://www.weap21.org/. You need to have a WEAP License. See here (https://www.weap21.org/index.asp?action=217) 


2.2 Register WEAP with Windows to allow the WEAP API to be accessed 
Use Windows "Command Prompt". Right click and then <font color=red>**run as Administrator**</font>, navigate to the WEAP installation directory such as and then hit enter  

```
cd C:\Program Files (x86)\WEAP
```

Then type the following command in the command prompt and hit enter   
```
WEAP /regserver
```

2.3 In some cases, you need to install this dependency 
•	You need to install pywin32 extensions which provide access to many of the Windows APIs from Python.

•	Install using an executable based on your python version. I used Python 2.7 https://github.com/mhammond/pywin32/releases
For advanced users, you can run the WaMDaM from source code at https://github.com/mhammond/pywin32




<img src="https://github.com/WamdamProject/WaMDaM-software-ecosystem/blob/master/mkdocs/Edit_MD_Files/QuerySelect/images/RegisterWEAP_CMD.png?raw=true" style="float:center;width:700px;padding:20px"> 
Figure 1: Register WEAP API with windows using the Command Prompt (Run as Administrator)


# Download the Bear River WEAP model

Clone or download all this GitHub repo   
https://github.com/WamdamProject/WaMDaM_UseCases  

In your local repo folder, go to the    
    
    C:\Users\Adel\Documents\GitHub\WaMDaM_UseCases/UseCases_files/1Original_Datasets_preperation_files/WEAP/Bear_River_WEAP_Model_2017

Copy this folder **Bear_River_WEAP_Model_2017** and paste it into **WEAP Areas** folder on your local machine. For example, it is at   

    C:\Users\Adel\Documents\WEAP Areas  


In [ ]:
# this library is needed to connect to the WEAP API
import win32com.client

# this command will open the WEAP software (if closed) and get the last active model
# you could change the active area to another one inside WEAP or by passing it to the command here
#WEAP.ActiveArea = "BearRiverFeb2017_V10.9"


WEAP=win32com.client.Dispatch("WEAP.WEAPApplication")

# WEAP.Visible = 'FALSE'


WEAP.ActiveArea = "Bear_River_WEAP_Model_2017"  

print 'Connected to WEAP API'
print '-------------'
if not WEAP.Registered:
    print "Because WEAP is not registered, you cannot use the API"

# get the active WEAP Area (model) to serve data into it 
ActiveArea=WEAP.ActiveArea.Name 
print  'ActiveArea= '+ActiveArea

# get the active WEAP scenario to serve data into it 
print '-------------'

ActiveScenario= WEAP.ActiveScenario.Name
print '\n ActiveScenario= '+ActiveScenario
print '-------------'

WEAP_Area_dir=WEAP.AreasDirectory
print WEAP_Area_dir


print "\n \n You're connected to the WEAP API"

<a name="CreateWEAP_Area"></a>
# C.1 Create a new WEAP Area for with new Reservoir Bathymetry


In [ ]:
# Create a copy of the WEAP AREA to serve the updated Hyrym Reservoir to it 

# UpdateHyrumBathym

WEAP.SaveAreaAs("Bear_River_WEAP_Model_2017_UpdateHyrumBathym")

WEAP.Scenarios.Add('UpdateHyrumBathym','Current Accounts')
# WEAP.Scenarios.Add('UpdateBearHeadFlowDry','Current Accounts')
WEAP.ActiveArea.Save
print 'A new WEA area has been created and a scenario is added to it'    
WEAP.ActiveArea = "Bear_River_WEAP_Model_2017"  
print  'ActiveArea= '+WEAP.ActiveArea.Name

WEAP.Quit

print 'Connection with WEAP API is disconnected'

<a name="AddScenarios"></a>
# 3. Add new scenarios from the Reference
 Add new Scenario
 Add(NewScenarioName, ParentScenarioName or Index): 
 Create a new scenario as a child of the parent scenario specified.  
 The new scenario will become the selected scenario in the Data View.  

In [ ]:
WEAP=win32com.client.Dispatch("WEAP.WEAPApplication")
# WEAP.Visible = FALSE


WEAP.ActiveArea = "Bear_River_WEAP_Model_2017"  

print  'ActiveArea= '+ WEAP.ActiveArea.Name

Scenarios=[]

Scenarios=['UpdateBearHeadFlowDry','UpdateCacheDemand']#,'UpdateHyrumBathym']


for scen in Scenarios:
    print scen
    WEAP.Scenarios.Add(scen,'Reference')
    
WEAP.ActiveArea.Save
WEAP.SaveArea

WEAP.Quit

# or add the scenarios one by one using this command   
    
# Make a copy from the reference (base) scenario
# WEAP.Scenarios.Add('UpdateCacheDemand','Reference')
print '----------------------'
print '\n Scenarios added to the original WEAP area'    

<a name="Connect"></a>
# 4. Connect to the WaMDaM populated SQLite file 


In [ ]:
# Then we can run queries against it within this notebook :)  

# the SQLite file is published here 
#https://github.com/WamdamProject/WaMDaM_UseCases/blob/master/UseCases_files/3SQLite_database/BearRiverDatasets_June_2018.sqlite

WaMDaM_SQLite_Name='BearRiverDatasets_August_2018.sqlite'


conn = sqlite3.connect(WaMDaM_SQLite_Name)

print 'Connected to the WaMDaM SQLite file called'+': '+ WaMDaM_SQLite_Name

<a name="QuerySupplyDataLoadWEAP"></a>
# A. Query Supply Flow TimeSeries Data and load it to WEAP 


<a name="QueryFlowData"></a>
# A.1 Query WaMDaM dababase for flow time series and prepare it for WEAP

In [ ]:
# Use Case 3.1Identify_aggregate_TimeSeriesValues.csv
# plot aggregated to monthly and converted to acre-feet time series data of multiple sources


# 2.2Identify_aggregate_TimeSeriesValues.csv
Query_UseCase3_1_URL="""
https://raw.githubusercontent.com/WamdamProject/WaMDaM_UseCases/master/4_Queries_SQL/UseCase3/UseCase3.1/2_Identify_aggregate_TimeSeriesValues.sql

"""

# Read the query text inside the URL
Query_UseCase3_1_text = urllib.urlopen(Query_UseCase3_1_URL).read()


# return query result in a pandas data frame
result_df_UseCase3_1= pd.read_sql_query(Query_UseCase3_1_text, conn)

# uncomment the below line to see the list of attributes
# display (result_df_UseCase3_1)


# Save the datafrom as a csv file into the Jupyter notebook working space
# result_df_UseCase3_1.to_csv('UseCases_Results_csv\UseCase3_1.csv', index = False)

df_TimeSeries=result_df_UseCase3_1


# Use Case 2.4_plotcdf 

# plot Cumulative flow for June for the UDWR dataset. 
# Then get the percentage of time it exceeds dry and wet years 

# Adel Abdallah
# July 31, 2018

## read the input data from GitHub csv file which is a direct query output for this  query:
# 3.2Identify_aggregate_TimeSeriesValues.sql

# Convert CalenderYear column data type to datetime
df_TimeSeries['CalenderYear'] = pd.to_datetime(df_TimeSeries['CalenderYear'], errors='coerce')

# Slice rows based on DatasetAcronym column
subsets = df_TimeSeries.groupby('ResourceTypeAcronym')

# Select rows where DatasetAcronym is UDWRFlowData
dt = subsets.get_group(name='UDWRFlowData')
months=[]
drySeasonal=[]
for m in range(1, 13):
    # From the selected rows, select rows where month is June
    specific_month = dt.CalenderYear.dt.month == m
    #  print specific_month[6]
    # CumulativeMonthly data of the desired DatasetAcronym name and month
    cumulative_monthly = dt[specific_month].CumulativeMonthly.values.tolist()

    # Sort cumulative_monthly in ascending order
    cumulative_monthly.sort()


    # Save the filtered data to csv, CumulativeMonthly and CalenderYear columns
    filtered_data = dt[specific_month][['CumulativeMonthly', 'CalenderYear']]
    # filtered_data.to_csv('Filtered Data.csv', index=False)
    

    p = np.percentile(cumulative_monthly, 5,interpolation="lower") # return 50th percentile, e.g median.
    drySeasonal.append(p)
    
    month=calendar.month_abbr[m]
    months.append(month)


d={'SeasonName':months,'SeasonNumericValue':drySeasonal}
df_Seasonal=pd.DataFrame(d)    

# display (df_Seasonal)
# SeasonName,SeasonNumericValue
SeasonalParam = ''
# print df_Seasonal['SeasonName']
for i in range(len(df_Seasonal['SeasonName'])):
#     if i<1:
#         continue 
    m_data = df_Seasonal['SeasonName'][i]
    n_data = df_Seasonal['SeasonNumericValue'][i]
    SeasonalParam += '{},{}'.format(m_data, n_data)
    if i != len(df_Seasonal['SeasonName']) - 1:
        SeasonalParam += ','

Seasonal_value="MonthlyValues("+SeasonalParam+")"



print Seasonal_value
print 'Query and data preperation are done'    

<a name="LoadFlow"></a>
# A2. Load the seasonal headflow data into WEAP

In [ ]:
# 9. Load the seasonal data into WEAP
WEAP=win32com.client.Dispatch("WEAP.WEAPApplication")
# WEAP.Visible = FALSE



WEAP.ActiveArea = "Bear_River_WEAP_Model_2017"  

print WEAP.ActiveArea.Name

WEAP.ActiveScenario = "UpdateBearHeadFlowDry"
print WEAP.ActiveScenario.Name


InstanceName='Central Bear'
AttributeName='Headflow'


# Get the Instance Name and Attribute names and pass them to 
# the function below to load their values into WEAP

# Seasonal_value contains the string of month value pairs 

for Branch in WEAP.Branches:
    if Branch.Name == InstanceName:
        GetInstanceFullBranch = Branch.FullName
        WEAP.Branch(GetInstanceFullBranch).Variable(AttributeName).Expression = Seasonal_value

print '\n The seasonal dry headflow data have been sucsesfully loaded into WEAP'

WEAP.SaveArea

print '\n \n The updated data have been saved'

<a name="QueryandLoadDemand"></a>
# B. Query demand data and load it to WEAP 

<a name="QueryDemand"></a>
# B1. Query Cache County Demand

In [ ]:
# 2.2Identify_aggregate_TimeSeriesValues.csv
Query_UseCase3_2c_URL="""
https://raw.githubusercontent.com/WamdamProject/WaMDaM_UseCases/master/4_Queries_SQL/UseCase3/UseCase3.2/2_IdentifyDemandSites_Seasonal_Numeric_totals.sql
"""

# Read the query text inside the URL
Query_UseCase3_2c_text = urllib.urlopen(Query_UseCase3_2c_URL).read()


# return query result in a pandas data frame
result_df_UseCase3_2c= pd.read_sql_query(Query_UseCase3_2c_text, conn)

# uncomment the below line to see the list of attributes
# display (result_df_required)

dfc=result_df_UseCase3_2c

# display (dfc)

# DataFrame.get_value(index, col, takeable=False)[source]

USU_WEAP_Model_2017_Numeric=dfc.get_value(1,'TotalNumeric')
# print USU_WEAP_Model_2017_Numeric

USU_WEAP_Model_2017_Seasonal=dfc.get_value(2,'TotalAnnualUseCacheCanals')
# print USU_WEAP_Model_2017_Seasonal

AnnualCacheCoDemand=USU_WEAP_Model_2017_Seasonal+ USU_WEAP_Model_2017_Numeric

print "AnnualCacheCoDemand=" 
print AnnualCacheCoDemand

#Max Delivered flow for Cache County as in the WaDE data source
WaDE_max=256430

# add the additional demand to West Cache Irr demand site as a multipler to the existing demand

Difference=WaDE_max-AnnualCacheCoDemand

WestCacheIrr_Dem=40521.82091

Multiplier=(Difference+WestCacheIrr_Dem)/WestCacheIrr_Dem
print "Multiplier="
print Multiplier
print '\n Query and data preperation are done'    

<a name="LoadDemand"></a>
# B2. Use WEAP API to load the demand data

In [ ]:
# Make a copy from the reference (base) scenario
# WEAP.Scenarios.Add('UpdateCacheDemand','Reference')
# WEAP.Areas("USU_WEAP_2017").Open
# Activate the scenario
WEAP.Scenarios("UpdateCacheDemand").Activate 

WEAP.ActiveScenario = "UpdateCacheDemand"

print WEAP.ActiveScenario.Name

# Update the WEAP Monthly Demand at West Cache Irr by adding the multiplier
for Branch in WEAP.Branches:
    for V in Branch.Variables:
        if Branch.Name == 'West Cache Irr' and V.Name== 'Monthly Demand':
            GetExpresValue = V.Expression
            print GetExpresValue
            SetExpresValue=GetExpresValue+'*2.03174927829'
            print SetExpresValue
            WEAP.Branch(Branch.FullName).Variable('Monthly Demand').Expression=0
            print V.Expression

            WEAP.SaveArea            
            WEAP.ActiveArea.Save

            WEAP.Branch(Branch.FullName).Variable('Monthly Demand').Expression=SetExpresValue
            WEAP.ActiveArea.Save
            WEAP.SaveArea 
            print 'Area is saved'
            print V.Expression

print ' \n \n Demand data loaded to WEAP'  

# WEAP.ActiveArea.Save
print 'Area is saved'
WEAP.SaveArea
print ' \n \n The updated data have been saved'

WEAP.Quit
print 'WEAP is saved and closed' 


<a name="QueryLoadBathymetry"></a>
# C. Query Bathymetry data and load it to WEAP 

<a name="QueryBathymetry"></a>
# C.2 Query the multi-column array: Reservoir Bathymetry and prepare it for WEAP


In [ ]:
# Use Case 3.1Identify_aggregate_TimeSeriesValues.csv
# plot aggregated to monthly and converted to acre-feet time series data of multiple sources


# display (df_Seasonal)


# column_name = "InstanceName"
# subsets = df_Seasonal.groupby(column_name)


# 4.3MergeTimeSeriesValues
Query_UseCase3_3b_URL="""
https://raw.githubusercontent.com/WamdamProject/WaMDaM_UseCases/master/4_Queries_SQL/UseCase3/UseCase3.3/4_MultiAttributeValues.sql

"""

# Read the query text inside the URL
Query_UseCase3_3b_text = urllib.urlopen(Query_UseCase3_3b_URL).read()


# return query result in a pandas data frame
result_df_UseCase3_3b= pd.read_sql_query(Query_UseCase3_3b_text, conn)

# uncomment the below line to see the list of attributes
# display (result_df_required)


# Save the datafrom as a csv file into the Jupyter notebook working space
# result_df_UseCase3_3b.to_csv('UseCases_Results_csv\result_df_UseCase3_3b.csv', index = False)

df=result_df_UseCase3_3b
# result_df_UseCase3_3b.to_csv('result_df_UseCase3_3b.csv', index = False)

# display (df)



# cur = conn.cursor()
# data = cur.execute(Query_UseCase3_3b_text)

# print data

# df = df.to_csv(header=True, index=False).strip('\n').split('\n')
# df=df.values.tolist()

# xx=df.to_csv(header=False, index=False)
# display (xx)

#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@


# 4.3MergeTimeSeriesValues
Query_UseCase3_3a_URL="""
https://raw.githubusercontent.com/WamdamProject/WaMDaM_UseCases/master/4_Queries_SQL/UseCase3/UseCase3.3/3_MergeTimeSeriesValues.sql

"""

# Read the query text inside the URL
Query_UseCase3_3a_text = urllib.urlopen(Query_UseCase3_3a_URL).read()


# return query result in a pandas data frame
result_df_UseCase3_3a= pd.read_sql_query(Query_UseCase3_3a_text, conn)

# result_df_UseCase3_3a.to_csv('result_df_UseCase3_3a.csv', index = False)


# uncomment the below line to see the list of attributes
# display (result_df_required)


# Save the datafrom as a csv file into the Jupyter notebook working space
# result_df_UseCase3_3a.to_csv('UseCases_Results_csv\result_df_UseCase3_3a.csv', index = False)

df2=result_df_UseCase3_3a
# df2 = df2.to_csv(header=True, index=False).strip('\n').split('\n')

# display (df2)

print '####################################'
print 'query is done'

import pandas as pd
df = pd.read_csv(
    
"""https://raw.githubusercontent.com/WamdamProject/WaMDaM_UseCases/master/5_Results_CSV/4.2MultiAttributeValues.csv""")
#print df
subsets = df.groupby('ScenarioName')

for subset in subsets.groups.keys():
    #print subset
    scenario_name_data = subsets.get_group(name=subset)
scenario_name_data = subsets.get_group(name='Base case')

df_MultiColumns=scenario_name_data

first_index = 0
first_order_value = df_MultiColumns['ValueOrder'][0]
second_index = 0
i = 0
n = 0

# print df_MultiColumns['ValueOrder']
for order_data in df_MultiColumns['ValueOrder']:

    if i == 0:
        i += 1
        continue
    if order_data == first_order_value:
        second_index = i
        n = second_index
        break
    i += 1

# diff = second_index - first_index
MultiParam = ''
# print n

for j in range(n):
    try:
        first_data = df_MultiColumns['Value'][j]
        second_data = df_MultiColumns['Value'][j + n]

        MultiParam += '{},{}'.format(second_data, first_data)
        if j != n - 1:
            MultiParam += ','
    except:
        break
# print MultiParam

# AttributeName(Value),AttributeName(Value)
# MultiParam=

csv_file_path_or_value_multi = "VolumeElevation(" + MultiParam + ")"
#print csv_file_path_or_value_multi

print 'VolumeElevation data is prepared'   


<a name="LoadBathymetry"></a>
# C3. Load the array data into WEAP

Add new Scenario
 Add(NewScenarioName, ParentScenarioName or Index): 
 Create a new scenario as a child of the parent scenario specified.  
 The new scenario will become the selected scenario in the Data View.  
 To delete a scenario, use the WEAPScenario method Delete (see below).

 Delete an scenario
 Delete(DeleteChildren): Delete this scenario.  
 To delete a scenario that has children, you must set the optional DeleteChildren parameter to True.

 WEAP.Scenarios("Larger reservoir").Delete(True)


 Activate a scenario
 Activate: Make this scenario the active scenario. 
 The active scenario will be used when accessing a WEAPVariable.Expression.

 WEAP.Scenarios("Current Accounts").Activate


In [ ]:
# 8. Load the array data into WEAP

WEAP=win32com.client.Dispatch("WEAP.WEAPApplication")
# WEAP.Visible = FALSE



print WEAP.ActiveArea.Name 
WEAP.ActiveArea = "Bear_River_WEAP_Model_2017_UpdateHyrumBathym"  
print WEAP.ActiveArea.Name 

WEAP.Areas("Bear_River_WEAP_Model_2017_UpdateHyrumBathym").Open
WEAP.ActiveArea = "Bear_River_WEAP_Model_2017_UpdateHyrumBathym"  
print WEAP.ActiveArea.Name

#Required_AttributeName
# Get the Instance Name and Attribute names and pass them to 
# the function below to load their values into WEAP

# the curve only exists in the current accounts 
# WEAP.Scenarios("Current Accounts").Activate
WEAP.ActiveScenario = "Current Accounts"
print WEAP.ActiveScenario.Name


InstanceName=df_MultiColumns['InstanceName'][1] #'Hyrum Reservoir'

#AttributeName = df_MultiColumns['MultiAttributeName'][2] # Volume Elevation Curve

# timeSeriesValue is the PATH for the cvs file for time series

AttributeName= 'Volume Elevation Curve'
for Branch in WEAP.Branches:
    if Branch.Name == InstanceName:
        GetInstanceFullBranch = Branch.FullName
        WEAP.Branch(GetInstanceFullBranch).Variable(AttributeName).Expression = csv_file_path_or_value_multi


AttributeName= 'Storage Capacity'
for Branch in WEAP.Branches:
    if Branch.Name == InstanceName:
        GetInstanceFullBranch = Branch.FullName
        WEAP.Branch(GetInstanceFullBranch).Variable(AttributeName).Expression = '17746'

print 'Elevation storage curve is loaded to WEAP'

WEAP.SaveArea

print '\n \n The updated data have been saved'

# print WEAP.ActiveArea.Name 
# WEAP.ActiveArea = "USU_WEAP_2017"  
# print WEAP.ActiveArea.Name 

# WEAP.Areas("USU_WEAP_2017").Open

# Run WEAP and report back the unment demand at the Bird Refuge demand site

<font color=green>**Please wait, it will take ~3-7 minutes** to finish calcualting the two WEAP Areas with their many scenarios</font>

In [ ]:
# Run WEAP

WEAP.Areas("Bear_River_WEAP_Model_2017").Open
print WEAP.ActiveArea.Name

WEAP.ActiveArea = "Bear_River_WEAP_Model_2017"  
print WEAP.ActiveArea.Name

WEAP.Calculate(2006,10,True)
WEAP.SaveArea

print '\n \n The calculation has been done and saved'
print WEAP.CalculationTime

UnmetDemandEstimate={}
Scenarios=[]
Scenarios=['Reference','UpdateBearHeadFlowDry','UpdateCacheDemand']
for scen in Scenarios:        
        print '\n-----------------------'
        print 'scenario='+ scen
#         print 'Tota Unmet Demand in 2002 for the Bird Refuge in Acre-feet='
       
        UnmetDemandEstimate[scen]=WEAP.ResultValue("\Demand Sites\Bird Refuge: Unmet Demand[Acre-Foot]", 2002, 1, scen, 2002, WEAP.NumTimeSteps) 

# print UnmetDemandEstimate
WEAP.SaveArea
WEAP.Quit        

WEAP.Areas("Bear_River_WEAP_Model_2017_UpdateHyrumBathym").Open
print WEAP.ActiveArea.Name

WEAP.ActiveArea = "Bear_River_WEAP_Model_2017_UpdateHyrumBathym"  
print WEAP.ActiveArea.Name

        
WEAP.Calculate(2006,10,True)
WEAP.SaveArea
print '\n \n The calculation for the WEAP area USU_WEAP_2017_UpdateHyrumBathym has been done and saved'

print 'scenario=UpdateHyrumBathym'

# print 'Total Unmet Demand in 2002 for the Bird Refuge in Acre-feet='        
UnmetUpdateHyrumBathym=WEAP.ResultValue("\Demand Sites\Bird Refuge: Unmet Demand[Acre-Foot]", 2002, 1, 'UpdateHyrumBathym', 2002, WEAP.NumTimeSteps) 


UnmetDemandEstimate.update({'UpdateHyrumBathym':UnmetUpdateHyrumBathym})
print '\n'
print UnmetDemandEstimate
# print 'Scenarios are deleted from WEAP'
WEAP.SaveArea


# plot the unmet demad for all the scenarios

In [ ]:


# xx=['Reference','Cache County <br> demand','Hyrum  <br> Bathymetry','Headflow <br> supply']


# yy=[Reference,UpdateDemand,UpdateBathem,UpdateHeadflows]
keys = UnmetDemandEstimate.keys()
vals = UnmetDemandEstimate.values()
keys,values = zip(*UnmetDemandEstimate.items())
print vals
print keys


ss= go.Scatter(
            x=keys,
            y=vals,
            mode = 'markers',
            marker = dict(
                size =15,
                color = '#0c0c0c',
                maxdisplayed=20,
                symbol ='circle',
                )
)

data=[ss]

layout = dict(
    #title = "Use Case 3.3",
    yaxis = dict(
        title = "Total unmet <br> demand in 2004 <br> (acre-feet))",
        tickformat= ',',
        showline=True,
        dtick='20000',
        ticks='outside',
        ticklen=10,
         range = ['0', '60000']


                ),
    
    xaxis = dict(
        title = "Updated input parameters <br>WEAP area USU_WEAP_2017",
        showline=True,
        ticks='outside',

        ticklen=10
                    ),
    legend=dict(
        x=0.1,y=0.6,
          bordercolor='#00000f',
            borderwidth=2
        
               ),
    width=1100,
    height=700,
    #paper_bgcolor='rgb(233,233,233)',
    #plot_bgcolor='rgb(233,233,233)',
    margin=go.Margin(l=300,b=200),
    font=dict(size=20)
             )
# create a figure object
fig = dict(data=data, layout=layout)
#py.iplot(fig, filename = "2.3Identify_SeasonalValues") 


## it can be run from the local machine on Pycharm like this like below
## It would also work here offline but in a seperate window  
offline.iplot(fig,filename = 'jupyter/UnmentDemand@BirdRefuge' )       



<a name="Close"></a>
# Close the SQLite and WEAP API connections

In [ ]:
# 9. Close the SQLite and WEAP API connections
conn.close()

print 'connection disconnected'

# Uncomment 
# WEAP.SaveArea

# Or 
# NewWEAPCopyName=ActiveArea+"Test"
# print NewWEAPCopyName

# Call API function to save WEAP
# WEAP.SaveAreaAS(NewWEAPCopyName)

# this command will close WEAP
# WEAP.Quit

print 'Connection with WEAP API is disconnected'

# The End :) Congratulations!

# (Optional..if needed) Delete exisitng scenarios
 Delete an scenario
 Delete(DeleteChildren): Delete this scenario.  
 To delete a scenario that has children, you must set the optional DeleteChildren parameter to True.

 WEAP.Scenarios("Larger reservoir").Delete(True)

In [ ]:
# Scenarios=[]
# Scenarios=['UpdateBearHeadFlowDry','UpdateCacheDemand']#,'UpdateHyrumBathym']
# for scen in Scenarios:
#     print 'scenario='+ scen
#     WEAP.Scenarios(scen).Delete(True)
# print '\n \n Scenarios are deleted from WEAP'